# Entrega Final Usando LigtGBM

\

In [ ]:
#descomprimir archivos
!ls
!unzip udea-ai4eng-20242.zip


sample_data  udea-ai4eng-20242.zip
Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
pip install lightgbm

## Entrega final modleo 1



```
# Tiene formato de código
```

# solucion 2
con 60% de datos


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
import lightgbm as lgb
from datetime import datetime
import os

class SaberProLightGBMFast:
    def __init__(self, sample_size=0.9):  # Por defecto usa 90% de los datos
        self.label_encoder = LabelEncoder()
        self.sample_size = sample_size
        self.pipeline = None

    def fit_transform_target(self, y):
        return self.label_encoder.fit_transform(y)

    def prepare_data(self, df):
        """
        Prepara los datos con mapeos simplificados
        """
        df_processed = df.copy()

        # Mapeos simplificados
        df_processed['FAMI_ESTRATOVIVIENDA'] = df_processed['FAMI_ESTRATOVIVIENDA'].fillna('Estrato 3').map({
            f"Estrato {i}": i for i in range(1, 7)
        })

        df_processed['ESTU_HORASSEMANATRABAJA'] = df_processed['ESTU_HORASSEMANATRABAJA'].fillna('0').map({
            '0': 0, 'Menos de 10 horas': 5, 'Entre 11 y 20 horas': 15,
            'Entre 21 y 30 horas': 25, 'Más de 30 horas': 35
        })

        # Simplificamos el mapeo de valor matrícula
        valor_matricula_map = {
            'Menos de 500 mil': 0.5,
            'Entre 500 mil y menos de 1 millón': 1,
            'Entre 1 millón y menos de 2.5 millones': 2,
            'Entre 2.5 millones y menos de 4 millones': 3,
            'Entre 4 millones y menos de 5.5 millones': 4,
            'Entre 5.5 millones y menos de 7 millones': 5,
            'Más de 7 millones': 6,
            'No pagó matrícula': 0
        }
        df_processed['ESTU_VALORMATRICULAUNIVERSIDAD'] = df_processed['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('Entre 1 millón y menos de 2.5 millones').map(valor_matricula_map)

        # Variables binarias
        df_processed['FAMI_TIENEINTERNET'] = df_processed['FAMI_TIENEINTERNET'].fillna('Si').map({'Si': 1, 'No': 0})
        df_processed['ESTU_PAGOMATRICULAPROPIO'] = df_processed['ESTU_PAGOMATRICULAPROPIO'].fillna('No').map({'Si': 1, 'No': 0})

        # Eliminar columnas no necesarias
        columns_to_drop = ['ID', 'PERIODO', 'FAMI_EDUCACIONPADRE',
                          'FAMI_EDUCACIONMADRE', 'ESTU_PRGM_ACADEMICO']
        df_processed = df_processed.drop(columns_to_drop, axis=1, errors='ignore')

        return df_processed

    def create_model(self):
        """
        Crea un pipeline simplificado con LightGBM
        """
        numeric_features = ['FAMI_ESTRATOVIVIENDA', 'ESTU_HORASSEMANATRABAJA',
                          'ESTU_VALORMATRICULAUNIVERSIDAD', 'FAMI_TIENEINTERNET',
                          'ESTU_PAGOMATRICULAPROPIO']

        categorical_features = ['ESTU_PRGM_DEPARTAMENTO']

        # Preprocessor simplificado
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), numeric_features),
                ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
            ])

        # Configuración básica de LightGBM
        lgb_params = {
            'objective': 'multiclass',
            'boosting_type': 'gbdt',
            'n_estimators': 100,
            'num_leaves': 31,
            'max_depth': 7,
            'learning_rate': 0.1,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'random_state': 42,
            'n_jobs': -1
        }

        # Pipeline simplificado
        self.pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', lgb.LGBMClassifier(**lgb_params))
        ])

        return self.pipeline

    def train_and_evaluate(self, X_train, y_train):
        """
        Entrena el modelo con validación cruzada simple
        """
        print("Tomando una muestra del", self.sample_size * 100, "% de los datos...")

        # Tomar una muestra del dataset
        X_sample, _, y_sample, _ = train_test_split(
            X_train, y_train,
            train_size=self.sample_size,
            stratify=y_train,
            random_state=42
        )

        print(f"Tamaño de la muestra de entrenamiento: {len(X_sample)} registros")

        # Crear y entrenar el modelo
        self.create_model()

        print("Entrenando modelo...")
        # Usar cross_val_score para una evaluación rápida
        scores = cross_val_score(
            self.pipeline, X_sample, y_sample,
            cv=3, scoring='f1_weighted', n_jobs=-1
        )

        print("\nResultados de la validación cruzada:")
        print(f"F1-weighted promedio: {scores.mean():.3f} (+/- {scores.std() * 2:.3f})")

        # Entrenar el modelo final con todos los datos de la muestra
        self.pipeline.fit(X_sample, y_sample)

        return scores.mean()

    def prepare_submission(self, test_df):
        """
        Prepara el archivo de submission
        """
        test_processed = self.prepare_data(test_df)
        predictions = self.pipeline.predict(test_processed)

        if hasattr(self, 'label_encoder') and self.label_encoder.classes_.size > 0:
            predictions = self.label_encoder.inverse_transform(predictions)

        submission = pd.DataFrame({
            'ID': test_df['ID'],
            'RENDIMIENTO_GLOBAL': predictions
        })

        return submission

    def save_submission(self, submission_df):
        """
        Guarda el archivo de submission
        """
        submission_dir = 'submissions'
        os.makedirs(submission_dir, exist_ok=True)

        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filepath = os.path.join(submission_dir, f'submission_lgbm_fast_{timestamp}.csv')

        submission_df.to_csv(filepath, index=False)

        print(f"\nSubmission guardada en: {filepath}")
        print("\nDistribución de predicciones:")
        print(submission_df['RENDIMIENTO_GLOBAL'].value_counts())

def main():
    print("Cargando datos...")
    train_df = pd.read_csv("train.csv")
    test_df = pd.read_csv("test.csv")

    X_train = train_df.drop(['RENDIMIENTO_GLOBAL'], axis=1)
    y_train = train_df['RENDIMIENTO_GLOBAL']

    # Crear pipeline con 30% de los datos
    pipeline = SaberProLightGBMFast(sample_size=0.3)

    # Codificar target
    y_train_encoded = pipeline.fit_transform_target(y_train)

    # Preparar datos
    print("Preparando datos...")
    X_train_processed = pipeline.prepare_data(X_train)

    # Entrenar y evaluar
    print("\nIniciando entrenamiento...")
    score = pipeline.train_and_evaluate(X_train_processed, y_train_encoded)

    # Generar predicciones
    print("\nGenerando predicciones...")
    submission = pipeline.prepare_submission(test_df)

    # Guardar submission
    pipeline.save_submission(submission)

    return pipeline, submission

if __name__ == "__main__":
    pipeline, submission = main()

Cargando datos...
Preparando datos...

Iniciando entrenamiento...
Tomando una muestra del 30.0 % de los datos...
Tamaño de la muestra de entrenamiento: 207750 registros
Entrenando modelo...

Resultados de la validación cruzada:
F1-weighted promedio: 0.384 (+/- 0.001)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010771 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 83
[LightGBM] [Info] Number of data points in the train set: 207750, number of used features: 32
[LightGBM] [Info] Start training from score -1.371986
[LightGBM] [Info] Start training from score -1.387094
[LightGBM] [Info] Start training from score -1.395026
[LightGBM] [Info] Start training from score -1.391226
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

Generando predicciones...

Submission guardada en: submissions/submission_lgbm_fast_2024